In [1]:
!pip install langchain
!pip install langchain_community
!pip install transformers
!pip install langchain_huggingface
!pip install faiss-cpu
!pip install sentence_transformers
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [10]:
from langchain.docstore.document import Document as langchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain.chains import RetrievalQA

In [11]:
import torch
device = 0 if torch.cuda.is_available() else -1
torch.cuda.is_available()

True

In [12]:
myData = {
    "about": [
        "Meet Galaxy S24 Ultra, the ultimate form of Galaxy Ultra with a new titanium exterior and a 17.25cm (6.8\") flat display. It's an absolute marvel of design.",
        "The legacy of Galaxy Note is alive and well. Write, tap and navigate with precision your fingers wish they had on the new, flat display.",
        "With the most megapixels on a smartphone and AI processing, Galaxy S24 Ultra sets the industry standard for image quality every time you hit the shutter. What's more, the new ProVisual engine recognizes objects — improving colour tone, reducing noise and bringing out detail.",
        "A new way to search is here with Circle to Search. While scrolling your fav social network, use your S Pen or finger to circle something and get Google Search results.",
        "Victory can be yours with the new Snapdragon 8 Gen 3 for Galaxy. Faster processing gives you the power you need for all the gameplay you want. Then, manifest graphic effects in real time with ray tracing for hyper-realistic shadows and reflections."
    ],
    "product_details": {
        "image": "https://m.media-amazon.com/images/I/711zU2HCovL._SX569_.jpg",
        "name": "Samsung Galaxy S24 Ultra 5G AI Smartphone (Titanium Black, 12GB, 256GB Storage)",
        "price": "93,990.",
        "rating": "4.0 out of 5 stars"
    },
    "reviews": [
        "Simply Superb and Awesome..!!!\nS24 Ultra is the best Andriod available out there. Overall Styling, Top notch Camera, Premium feel, Titanium body, Super Advanced Processor, 12GB RAM, Vast Storage capacity (starting from 256GB), Best in class Battery backup, Reliable brand are the key USP's one should consider in this phone. I use IP15 Pro as my Primary phone and after this in hand, honestly iPhone 15 Pro feels outdated..!! Guys, if you are thinking of this phone, then just go for it..! No second thoughts.\n",
        "I recently upgraded to the Samsung Galaxy S24 Ultra, and I couldn't be happier! From the moment I unboxed it, the sleek design and premium feel set a new standard. The screen is stunning – bright, vibrant, and incredibly responsive, making streaming and gaming a real pleasure.\n\nPerformance-wise, this phone is a beast. Multitasking is a breeze, and apps open instantly, thanks to the top-tier processor. Battery life is also fantastic, easily lasting me a full day with heavy use, and fast charging means I'm never stuck waiting long when I need a boost.\n\nThe camera quality is next-level! Low-light photos come out crisp, the zoom is mind-blowing, and colors are captured beautifully. It's like having a professional camera in my pocket.\n\nOverall, the Galaxy S24 Ultra is worth every penny. It's a true flagship device that feels future-proof, and I'd highly recommend it to anyone looking for a powerful, feature-packed smartphone!\n",
        "The phone itself myt be one of thr best phones I have used till date but the problem is that the phone I received is of a different region(software) and not India so it's got defective issues. On contacting amazon service center and samsung service center I keep getting no solutions to my problem and will not allow a replacement since it's a software issue that cannot be solved. Samsung service center keeps holding my phone till the validity of return period and also saying they won't be able to give me the documents to return phone via amazon since their validity of the giving the documents have ended as it was counted from invoice date (date of ordering ) I received the phone after 5 days and have contacted both service centres within 2 days. I'm pretty much helpless now.\n",
        "Best android phone in the market at present.Awesome built and superb camera quality .Happy with the purchase.\n",
        "Excellent phone the camera quality is exceptional. Sound quality and battery life is good. Charging speed is fast.\n",
        "Amazing display, Great cam, charging speed, always cool.\nCons:\nIt doesn't maintain a steady video frame rate in low light.\nWireless 15w charging is slow\n\nMy rant on Apple\nI was interested to get the Apple watch but was hesitant to get the iphone but finally gave in and... IT WAS HORROR...\n\nWorst part, during exchange, you don't have access to your iphone so say bye bye to your WhatsApp backup... I had to get an old iphone from a friend to get the data transfered...\n\nAll that aside, S24 U has been just the right device for me...\n",
        "Very good phone,display and camera superb, smoth processor\n",
        "Works really well and good picture quality and Great zoom quality as well.\n",
    ]
}


In [13]:
def split_documents(chunk_size, raw_knowledge_base, tokenizer_name):
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=[
            "\n#{1,6}", "```\n", "\n\\\\\\*+\n", "\n---+\n", "\n_+\n", "\n\n", "\n", " ", ""
        ],
    )
    docs_processed = []
    for doc in raw_knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)
    return docs_processed_unique


In [ ]:
RAW_KNOWLEDGE_BASE = [langchainDocument(page_content=about) for about in myData["about"]]
RAW_KNOWLEDGE_BASE += [langchainDocument(page_content=review) for review in myData["reviews"]]
RAW_KNOWLEDGE_BASE += [langchainDocument(page_content="name : " + myData["product_details"]["name"])]
RAW_KNOWLEDGE_BASE += [langchainDocument(page_content= "price : " + myData["product_details"]["price"])]
RAW_KNOWLEDGE_BASE += [langchainDocument(page_content= "Rating : " + myData["product_details"]["rating"])]
RAW_KNOWLEDGE_BASE += [langchainDocument(page_content= "Image link : " + myData["product_details"]["image"])]

docs_processed = split_documents(512, RAW_KNOWLEDGE_BASE, "thenlper/gte-small")

embedding_model = HuggingFaceEmbeddings(
    model_name="thenlper/gte-small",
    model_kwargs={'device': 'cuda:0'},
    encode_kwargs={"normalize_embeddings": True} # this normalizes the embeddings between -1 and 1. This is done to use cosine distance similariity search
)


In [ ]:
for doc in docs_processed:
    print(doc.page_content)
    print("---")

In [15]:
vec_embeddings = embedding_model.embed_documents([doc.page_content for doc in docs_processed])
len(vec_embeddings[0]),len(docs_processed) ,embedding_model.model_name,embedding_model.encode_kwargs,embedding_model.cache_folder

(384, 17, 'thenlper/gte-small', {'normalize_embeddings': True}, None)

In [16]:
db = FAISS.from_documents(
    docs_processed,
    embedding_model,
    distance_strategy=DistanceStrategy.COSINE
)

In [ ]:
# !ls /root/.cache/huggingface/hub
# from transformers import GPTNeoForCausalLM, GPT2Tokenizer, pipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "HuggingFaceH4/zephyr-7b-beta"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model_name = "EleutherAI/gpt-neo-1.3B"
# model = GPTNeoForCausalLM.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)


# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)



# llm = pipeline(model=model,
#                tokenizer=tokenizer,
#                task="text-generation",
#                do_sample = True,
#                temperature = 0.2,
#                repitition_penality=1.1,
#                return_full_text=False,
#                max_new_tokens=500)


In [14]:
#  prompt_chat=[
#         {
#             "role":"system",
#             "content":"""Using the information of people's reviews contained in the context,
#     Give a comprehensive answer to the question.
#     Respond only to the question asked , response should be concise and relevant to the question.
#     If the answer cannot be deduced from the context, do not give an answer""",
#         },
#         {
#             "role":"user",
#             "content":"""Context:
#     {context}
#     ---
#     Now here is the Question you need to answer.
#     Question:{question}
#             """,
#         },
#     ]

# rag_template = tokenizer.apply_chat_template(prompt_chat, tokenize=False,add_generation_prompt=True)
# rag_template


In [ ]:
# user_query = "Give me the highlights of S24 Ultra."
# retrieved_docs = db.similarity_search(user_query,k=2)
# for docs in retrieved_docs :
#   print(docs.page_content)

# rd_text = [doc.page_content for doc in retrieved_docs]
# context = "\n Extracted Documents : \n"
# context += "".join(f"Document {str(i)}:::\n" + doc for i,doc in enumerate(rd_text))

In [22]:

# prompt = f"""Using the information of people's reviews contained in the context,
# Give a comprehensive answer to the question.
# Respond only to the question asked, response should be concise and relevant to the question.
# If the answer cannot be deduced from the context, do not give an answer.

# Context:
# {context}
# ---
# Now here is the Question you need to answer.
# Question: {user_query}
# """


# # final_promt = rag_template.format(question=user_query,context=context)
# # print(final_promt)

In [ ]:
# response = llm(prompt)[0]["generated_text"]
# response

In [ ]:
from huggingface_hub import InferenceClient
import os
hf_token = os.getenv("HF_TOKEN")
client = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    token=hf_token
)

user_query = "price"
user_query = "Ho much it would take me to buy brand new S24 Ultra?"
user_query = "Ho much it would take me to buy brand new S24 Ultra, its price?"
retrieved_docs = db.similarity_search(user_query, k=7)

rd_text = [doc.page_content for doc in retrieved_docs]
context = "\nExtracted Documents:\n"
context += "".join(f"\n{doc}\n" for i, doc in enumerate(rd_text))
# context += "".join(f"Document {i}:\n{doc}\n" for i, doc in enumerate(rd_text))

prompt = f"""Using the information of people's reviews and product features contained in the context,
Give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
If the answer cannot be deduced from the context, do not give an answer(Do not dirctly say that there is no context  or no information, but say politely).

Context:
{context}
---
Now here is the Question you need to answer.
Question: {user_query}
"""

messages = [{"role": "user", "content": prompt}]
completion = client.chat.completions.create(
        messages=messages,
        max_tokens=500,
        temperature = 0.3

    )
response = completion.choices[0].message["content"]
response



' The price for the brand new Samsung Galaxy S24 Ultra 5G AI Smartphone (Titanium Black, 12GB, 256GB Storage) is not explicitly stated in the provided context. However, it can be inferred that it is a high-end device, and its price would typically be on the higher end of the smartphone market. For a more accurate price, it would be best to check the official Samsung website or a reliable electronics retailer.'